<a href="https://colab.research.google.com/github/ABmaxplunck/PyTorch-Codes-/blob/main/02_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear Regression one of the foundational algorithm of machine learning.
we will create a model that preducts crop yields for apples and oranges (target variables) by looking at the average temparature, rainfall and humidity (input variables or features) in a region.

---
NB: For finding documentation we can put a question mark then write the specific topic. as an example:

 ?nn.Linear



In [72]:
import numpy as np
import torch

### Training data
The training data can be represented using 2 matrices: inputs and targets, each with one row per observation and one column per variable

In [73]:
  # input(temp,rainfall,humidity)
  
  inputs = np.array([[73,67,43],
                      [91,88,64],
                      [87,134,58],
                      [102,13,37],
                      [69,96,70]], dtype='float32')

In [74]:
# Targets(apples, oranges)
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]], dtype='float32')

we've seperated the input and target variables, because we'll operate on them seperately. Also, we've created numpy arrays, because this is typically how you would work with training data:read some CSV files as numpy arrays, do some processing and then convert them to Pytorch tensors as follows:

In [75]:
# convert input and targets to tensors 
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  13.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Linear regression model from scratch
The weights and biases (w11, w12,...., w23, b1 & b2) cann also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable i.e: yield of apples and similarly the second for oranges.

In [76]:
# weigts and biases 

w = torch.randn(2, 3, requires_grad=True) # it has 2 row and 3 col
b = torch.randn(2, requires_grad=True) # it has 2 row  
print(w)
print(b)


tensor([[-0.7912,  0.3821, -1.7614],
        [-0.9419, -0.9434,  1.0948]], requires_grad=True)
tensor([-0.2898,  0.7894], requires_grad=True)


we can define the model as follows:


In [77]:
def model(x):
  return x @ w.t() + b

`@` represent the matrix multiplication in Pytorch and the `.t` method returns the transpose of a tensor

The matrix obtained by passing the input data into the model is a set of predictions for the target variables

In [78]:
# Generate predictions
preds = model(inputs)
print(preds)


tensor([[-108.1880,  -84.1044],
        [-151.3951,  -97.8805],
        [-120.0831, -144.0794],
        [-141.2013,  -67.0430],
        [-141.4989,  -78.1374]], grad_fn=<AddBackward0>)


let's compare the predictions of our model with the actual targets

In [79]:
# compare with targets 
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


you can see that there's a huge differnce between the predictions of our model and the actual values of the target variables. Obviously this is because we've initialized our model with random weights and biases and we can't expect it to just work 

### Loss Function
Before we improve our model, we need a way to evaluate how well our model is performing. we can compare the model's precitions with the actual targets, using the following method:

* calculate the difference between the two matrices (preds and targets)
* square all elements of the difference matrix to remove negative values 
* calculate the average of the elements in the resulting matrix

The result is a single number known as the `mean squared error (MSE)`

In [80]:

diff = preds - targets
diff_sqr = diff * diff # element wise multiplication 
torch.sum(diff_sqr) /  diff.numel()  # using .numel method to get the num of element 

tensor(41430.2930, grad_fn=<DivBackward0>)

In [81]:
# MSE loss function

def mse(t1,t2):
  diff = t1 -t2
  return torch.sum(diff * diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model 

In [82]:
# compute loss
loss = mse(preds, targets)
print(loss)



tensor(41430.2930, grad_fn=<DivBackward0>)


here's how we can interpret the result: On average each element in the prediction differs from the actual target by about 138(square root of the loss 19044). And that's pretty bad, considerting the numbers we are trying to predict are themselves in the range 50-200. Also, the result is called the loss, because it indicates how bad the model is at preciting the target variables. Lower the loss better the model 

### Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t to the weights and biases, because they have requires_grad set to True

In [83]:
# Compute gradients 

loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t the weights matrix is itself a matrix, with the same dimensions 

In [84]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.7912,  0.3821, -1.7614],
        [-0.9419, -0.9434,  1.0948]], requires_grad=True)
tensor([[-17490.1738, -17816.4023, -11790.7119],
        [-15533.7021, -17046.5723, -10614.9307]])


`w` is a weight matrix or tensor.
I have each corresponding weights and derivative of the loss

from `print(w)` we get losses which are w11, w12 etc etc 

from `print(w.grad)` we get derivative of the loss element. these are d loss by dw11, this is d loss by dw12 etc etc 

In [85]:
# For bias 
print(b)
print(b.grad)

tensor([-0.2898,  0.7894], requires_grad=True)
tensor([-208.6733, -186.2489])




> Indented block


The loss is a `quadratic function` of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like figureshown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the graph or loss function w.r.t the weights and biases

If a gradient element is `positive` 
* increasing the element's value slightly will increase the loss

* decreasing the element's value slightly will decrease the loss

If the gradient element is `negative` 
* increasing the element's value slightly will decrease the loss

* decreasing the element's value slightly will increase the loss



Before we proceed we reset the gradients to zero by calling `.zero_()` method. we need to do this, because PyTorch accumulates gradients i.e: the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results 

In [86]:

w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Adjust weights and biases using gradient descent
we'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the follwing steps:
1. Generate predictions
2. Calculate the loss
3. compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient 
5. Reset the gradients to zero

In [87]:
# Generate predictions

preds = model(inputs)
print(preds)

tensor([[-108.1880,  -84.1044],
        [-151.3951,  -97.8805],
        [-120.0831, -144.0794],
        [-141.2013,  -67.0430],
        [-141.4989,  -78.1374]], grad_fn=<AddBackward0>)


note that the predictions are same as before, since we haven't made any changes to our model. the same hols true for the loss and gradients

In [88]:
# calculate the loss

loss = mse(preds, targets)
print(loss)

tensor(41430.2930, grad_fn=<DivBackward0>)


In [89]:
# compute gradients

loss.backward()
print(w.grad)
print(b.grad)

tensor([[-17490.1738, -17816.4023, -11790.7119],
        [-15533.7021, -17046.5723, -10614.9307]])
tensor([-208.6733, -186.2489])


finally we update the weights and  biases using the gradients computed above 

In [90]:
# adjust weights and reset gradients 

with torch.no_grad():  # torch.no_grad() is way of telling pytorch that hey I'm done with my gradient calculations now let me use the gradients  
  w -= w.grad * 1e-5    # when I am running this operations don't track gradient related work 
  b -= b.grad * 1e-5
  w.grad.zero_() # reset the gradient back to zero
  b.grad.zero_()


a few things to note above:
* we use torch.no_grad to indicate PyTorch that we shouln't track, calculate or modify gradients while updating the weights and biases
* we multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to takse a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm 
* after we have updated the weights, we reset the gradients back to zero to avoid affecting any future computations 

Let's take a look at the new weights and biases 

In [91]:
print(w)
print(b)

tensor([[-0.6163,  0.5603, -1.6435],
        [-0.7866, -0.7730,  1.2009]], requires_grad=True)
tensor([-0.2877,  0.7913], requires_grad=True)


with the new weights and biases, the model should have lower loss

In [92]:
# calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(28565.5059, grad_fn=<DivBackward0>)


we have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent 

## Train for multiple epochs

To reduce the loss further we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. let's train the model for 100 epochs

In [93]:
# Train for 100 epochs

for i in range (100):
  preds = model(inputs) # get the predictions passing the inputs 
  loss = mse(preds, targets) # get the loss by passing the predctions and targets into the mse function
  loss.backward() # calculcate the gradients 
  with torch.no_grad(): #telling pytorch hey stop tracking gradient for a sec
    w -= w.grad * 1e-5 # here 1e-5 is learning rate which is called hyper parameter in ML
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()


once again, let's verify that the loss is now lower:

In [94]:
# calcualte lose
preds = model(inputs)
loss = mse(preds, targets)
print(loss)


tensor(191.3603, grad_fn=<DivBackward0>)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets 

In [95]:
# precitions 
preds

tensor([[ 66.5155,  73.6880],
        [ 78.7352, 109.9358],
        [140.1239, 113.7696],
        [ 14.8220,  42.4439],
        [ 75.6948, 126.8542]], grad_fn=<AddBackward0>)

In [96]:
# targets 
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs.

At this point, we can save our notebook and upload it to the git for future reference and sharing 


# Linear Regression using PyTorch built-ins
The model and training process above were implemented using basic matrix operations. But since this such a common pattern, PyTorch has several built in functions and classes to make it easy to create and train models 

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks 

In [97]:
import torch.nn as nn

as before we represent the inputs and targets and matrices 

In [98]:
# Input (temp, rainfall, humidity)

# here 15 training examples
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58],
                   [102, 43, 37], [69, 96,70], [73,67,43],
                   [91, 88, 64], [87, 134, 58], [102, 43, 37],
                   [69, 96, 70], [73, 67, 43], [91, 88, 64],
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56,70], [81,101], [119,133],
                    [22, 37], [103,119], [56, 70],
                    [81, 101], [119,133], [22, 37],
                    [103, 119], [56,70], [81, 101],
                    [119, 113], [22, 37], [103, 119]],
                   dtype='float32')

# convert into pyTorch tensors 
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


we are using 15 training examples this time, to illustrate how to work with large datasets in small batches.


NB:
Multiplying very large matrix has very high cost because it has very high time complexity like O(n^3). so it is better to work with smaller matrix. not more than 1000 rows or cols 

## Dataset and DataLoader
we will create a `TensorDataSet`, which allows access to rows from `inputs` and `targets` as tuples and provides standard APIs for working with many differnt types of datasets in PyTorch

In [99]:

from torch.utils.data import TensorDataset


In [100]:
# Define Dataset

train_ds = TensorDataset(inputs, targets)
train_ds[[1, 3, 5, 7]]

(tensor([[ 91.,  88.,  64.],
         [102.,  43.,  37.],
         [ 73.,  67.,  43.],
         [ 87., 134.,  58.]]), tensor([[ 81., 101.],
         [ 22.,  37.],
         [ 56.,  70.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation([0:3]) in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the slected rows, and the second contains the targets

------

we'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data 


In [101]:
from torch.utils.data import DataLoader

In [102]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

# when we set shuffle is equal true before creating batches it shuffle the data and then it creates batches after the shuffling 

The data loader is typically used in a for-in loop. Let's look at an example 

In [103]:
for xb, yb in train_dl:
  
  print(xb)
  print(yb)
  break

tensor([[102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.]])
tensor([[ 22.,  37.],
        [119., 133.],
        [119., 133.],
        [ 56.,  70.],
        [ 81., 101.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the trainng data before creating batches. Shuffling hels randomize the input to the optimization algorithm, which can lead to faster reduction in the loss

#**nn.Linear**
instead of initializing the weights and biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically that means initializing automatically 

In [104]:
# Define model 
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2186, -0.1735, -0.0878],
        [ 0.1846,  0.3620,  0.0760]], requires_grad=True)
Parameter containing:
tensor([0.4508, 0.3274], requires_grad=True)


PyTorch models also have a helpful parameters method, which returns a list containing all the weightsd and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [105]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2186, -0.1735, -0.0878],
         [ 0.1846,  0.3620,  0.0760]], requires_grad=True),
 Parameter containing:
 tensor([0.4508, 0.3274], requires_grad=True)]

we can use the model to generate predictions in the exact same way as before:

In [106]:
# Generate Predictions
preds = model(inputs)
preds

tensor([[-30.9060,  41.3269],
        [-40.3274,  53.8485],
        [-46.9069,  69.3059],
        [-32.5559,  37.5359],
        [-37.4321,  53.1395],
        [-30.9060,  41.3269],
        [-40.3274,  53.8485],
        [-46.9069,  69.3059],
        [-32.5559,  37.5359],
        [-37.4321,  53.1395],
        [-30.9060,  41.3269],
        [-40.3274,  53.8485],
        [-46.9069,  69.3059],
        [-32.5559,  37.5359],
        [-37.4321,  53.1395]], grad_fn=<AddmmBackward0>)

# **Loss Function**
instead of defining a loss function manually, we can use the built in loss function mse_loss

In [107]:
# Import nn.functional 
import torch.nn.functional as F

the nn.functional package conains many useful loss functions and several other utilities

In [108]:
# Define loss function
loss_fn = F.mse_loss